In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import numpy as np

# **웹 스크래핑**

In [ ]:
!pip install requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install beautifulsoup4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import requests
import csv
from bs4 import BeautifulSoup

In [ ]:
filename = "공모주.csv"
f = open(filename, "w", encoding="utf-8-sig", newline="")
writer = csv.writer(f)

name_list = []
competition_rate_list = []

for page in range(2,41):
  url = f"https://www.38.co.kr/html/fund/index.htm?o=nw&page={page}"
  res = requests.get(url)
  res.raise_for_status()
  soup = BeautifulSoup(res.text, "html.parser")

  table = soup.find("table", attrs={"summary":"신규상장종목"})

  if (page == 2):
    titles = table.find("thead").find_all("th")
    titles = [title.get_text() for title in titles]

    # csv file에 title header 삽입
    writer.writerow(titles[:-1])
  
  rows = table.find("tbody").find_all("tr")

  for row in rows:
    columns = row.find_all("td")

    # 기업명	신규상장일	현재가(원)	전일비(%)	공모가(원)	공모가대비 등락률(%)	시초가(원)	시초/공모(%)	첫날종가(원)
    data = [column.get_text().strip() for column in columns[:-1]]
    writer.writerow(data)

  # --------------------------------------------------------------------------------------------
  competition_url = f"https://www.38.co.kr/html/fund/index.htm?o=r1&page={page}"
  res = requests.get(competition_url)
  res.raise_for_status()
  soup = BeautifulSoup(res.text, "html.parser")
  table = soup.find("table", attrs={"summary":"수요예측결과"})
  rows = table.find("tbody").find_all("tr")
  
  for row in rows:
    columns = row.find_all("td")
    
    # 기업명	예측일	공모희망가(원)	공모가(원)	공모금액 (백만원)	기관 경쟁률	의무보유 확약	주간사
    data = [column.get_text().strip() for column in columns]
  
    name_list.append(data[0])
    competition_rate_list.append(data[5])

In [ ]:
stock_info_dataframe = pd.read_csv("공모주.csv", index_col = '기업명')
stock_info_dataframe

,신규상장일,현재가(원),전일비(%),공모가(원),공모가대비등락률(%),시초가(원),시초/공모(%),첫날종가(원)
기업명,,,,,,,,
마스턴프리미어리츠(유가),2022/05/31,"5,010",0.00%,"5,000",0.2%,"5,750",15%,"6,010"
하나금융스팩22호,2022/05/26,"2,140",0.94%,"2,000",7%,"2,160",8%,"2,155"
가온칩스,2022/05/20,"22,800",-3.80%,"14,000",62.86%,"24,150",72.5%,"27,100"
대명에너지,2022/05/16,"12,550",-5.28%,"15,000",-16.33%,"15,450",3%,"13,950"
상상인스팩3호,2022/04/29,"2,115",-0.24%,"2,000",5.75%,"2,155",7.75%,"2,150"
...,...,...,...,...,...,...,...,...
아이티센,2014/12/23,-,%,"7,500",%,"8,460",12.8%,"8,770"
제일모직(구.삼성에버랜드)(유가),2014/12/18,-,%,"53,000",%,"106,000",100%,"113,000"
하이로닉,2014/12/17,-,%,"51,000",%,"63,800",25.1%,"73,300"


In [ ]:
competition_rate_dataframe = pd.DataFrame(data=competition_rate_list, index = name_list, columns = ['기관경쟁률'])
competition_rate_dataframe

,기관경쟁률
마스턴프리미어리츠(유가),
대명에너지,254.74:1
상상인스팩3호,
포바이포,1846.32 :1
신영스팩7호,
...,...
슈피겐코리아,255.6:1
테라셈,18.41:1
영우디에스피,17.07:1
대우스팩2호,


In [ ]:
stock_info_dataframe = stock_info_dataframe.drop(['하나금융스팩22호', '가온칩스', '대신밸런스스팩10호', '신한알파리츠(유가)', '모두투어리츠(유가)'])
stock_info_dataframe['기관경쟁률'] = competition_rate_dataframe.loc[stock_info_dataframe.index]

In [ ]:
stock_info_dataframe

,신규상장일,현재가(원),전일비(%),공모가(원),공모가대비등락률(%),시초가(원),시초/공모(%),첫날종가(원),기관경쟁률
기업명,,,,,,,,,
마스턴프리미어리츠(유가),2022/05/31,"5,010",0.00%,"5,000",0.2%,"5,750",15%,"6,010",
대명에너지,2022/05/16,"12,550",-5.28%,"15,000",-16.33%,"15,450",3%,"13,950",254.74:1
상상인스팩3호,2022/04/29,"2,115",-0.24%,"2,000",5.75%,"2,155",7.75%,"2,150",
신영스팩7호,2022/04/28,"2,170",0.93%,"2,000",8.5%,"2,295",14.75%,"2,270",
포바이포,2022/04/28,"23,750",13.10%,"17,000",39.71%,"34,000",100%,"44,200",1846.32 :1
...,...,...,...,...,...,...,...,...,...
아이티센,2014/12/23,-,%,"7,500",%,"8,460",12.8%,"8,770",54.13:1
제일모직(구.삼성에버랜드)(유가),2014/12/18,-,%,"53,000",%,"106,000",100%,"113,000",465:1
하이로닉,2014/12/17,-,%,"51,000",%,"63,800",25.1%,"73,300",500.56:1


In [ ]:
# 가격 데이터 처리
condition = stock_info_dataframe[stock_info_dataframe['첫날종가(원)'] == '-'].index
stock_info_dataframe = stock_info_dataframe.drop(condition)

In [ ]:
stock_info_dataframe

,신규상장일,현재가(원),전일비(%),공모가(원),공모가대비등락률(%),시초가(원),시초/공모(%),첫날종가(원),기관경쟁률
기업명,,,,,,,,,
마스턴프리미어리츠(유가),2022/05/31,"5,010",0.00%,"5,000",0.2%,"5,750",15%,"6,010",
대명에너지,2022/05/16,"12,550",-5.28%,"15,000",-16.33%,"15,450",3%,"13,950",254.74:1
상상인스팩3호,2022/04/29,"2,115",-0.24%,"2,000",5.75%,"2,155",7.75%,"2,150",
신영스팩7호,2022/04/28,"2,170",0.93%,"2,000",8.5%,"2,295",14.75%,"2,270",
포바이포,2022/04/28,"23,750",13.10%,"17,000",39.71%,"34,000",100%,"44,200",1846.32 :1
...,...,...,...,...,...,...,...,...,...
아이티센,2014/12/23,-,%,"7,500",%,"8,460",12.8%,"8,770",54.13:1
제일모직(구.삼성에버랜드)(유가),2014/12/18,-,%,"53,000",%,"106,000",100%,"113,000",465:1
하이로닉,2014/12/17,-,%,"51,000",%,"63,800",25.1%,"73,300",500.56:1


In [ ]:
# 기관 경쟁률 데이터 처리
stock_info_dataframe = stock_info_dataframe.loc[stock_info_dataframe['기관경쟁률'].dropna().index]
stock_info_dataframe['기관경쟁률'] = stock_info_dataframe['기관경쟁률'].map(lambda x: (x[:x.find(':')].strip()))

In [ ]:
stock_info_dataframe

,신규상장일,현재가(원),전일비(%),공모가(원),공모가대비등락률(%),시초가(원),시초/공모(%),첫날종가(원),기관경쟁률
기업명,,,,,,,,,
마스턴프리미어리츠(유가),2022/05/31,"5,010",0.00%,"5,000",0.2%,"5,750",15%,"6,010",
대명에너지,2022/05/16,"12,550",-5.28%,"15,000",-16.33%,"15,450",3%,"13,950",254.74
상상인스팩3호,2022/04/29,"2,115",-0.24%,"2,000",5.75%,"2,155",7.75%,"2,150",
신영스팩7호,2022/04/28,"2,170",0.93%,"2,000",8.5%,"2,295",14.75%,"2,270",
포바이포,2022/04/28,"23,750",13.10%,"17,000",39.71%,"34,000",100%,"44,200",1846.32
...,...,...,...,...,...,...,...,...,...
아이티센,2014/12/23,-,%,"7,500",%,"8,460",12.8%,"8,770",54.13
제일모직(구.삼성에버랜드)(유가),2014/12/18,-,%,"53,000",%,"106,000",100%,"113,000",465
하이로닉,2014/12/17,-,%,"51,000",%,"63,800",25.1%,"73,300",500.56


In [ ]:
path = '/content/gdrive/Shareddrives/CS360/madcamp-3-week/data/strategy-2'
stock_info_dataframe.to_csv(path + '/strategy-2-data.csv', encoding='utf-8-sig')

# **데이터 분석**

## 모든 종목


In [ ]:
FILE_PATH = '/content/gdrive/Shareddrives/CS360/madcamp-3-week/data/strategy-2/strategy-2-data.csv'

In [ ]:
stock_info_dataframe = pd.read_csv(FILE_PATH, index_col = '기업명', thousands=',')
stock_info_dataframe = stock_info_dataframe.loc[:, ['공모가(원)', '시초가(원)', '첫날종가(원)']]

In [ ]:
stock_info_dataframe

,공모가(원),시초가(원),첫날종가(원)
기업명,,,
마스턴프리미어리츠(유가),5000,5750,6010
대명에너지,15000,15450,13950
상상인스팩3호,2000,2155,2150
신영스팩7호,2000,2295,2270
포바이포,17000,34000,44200
...,...,...,...
아이티센,7500,8460,8770
제일모직(구.삼성에버랜드)(유가),53000,106000,113000
하이로닉,51000,63800,73300


In [ ]:
stock_info_dataframe['시초가/공모가(%)'] = (stock_info_dataframe['시초가(원)']/stock_info_dataframe['공모가(원)'] - 1)*100
stock_info_dataframe['첫날종가/공모가(%)'] = (stock_info_dataframe['첫날종가(원)']/stock_info_dataframe['공모가(원)'] - 1)*100
stock_info_dataframe

,공모가(원),시초가(원),첫날종가(원),시초가/공모가(%),첫날종가/공모가(%)
기업명,,,,,
마스턴프리미어리츠(유가),5000,5750,6010,15.000000,20.200000
대명에너지,15000,15450,13950,3.000000,-7.000000
상상인스팩3호,2000,2155,2150,7.750000,7.500000
신영스팩7호,2000,2295,2270,14.750000,13.500000
포바이포,17000,34000,44200,100.000000,160.000000
...,...,...,...,...,...
아이티센,7500,8460,8770,12.800000,16.933333
제일모직(구.삼성에버랜드)(유가),53000,106000,113000,100.000000,113.207547
하이로닉,51000,63800,73300,25.098039,43.725490


### 청약 후 상장 당일 시가 매도

In [ ]:
ret = stock_info_dataframe['시초가/공모가(%)']

tgt_cnt = len(ret.index)
win_case_dataframe = ret[ret >= 0]
lose_case_dataframe = ret[ret < 0]
win_cnt = len(win_case_dataframe.index)
lose_cnt = len(lose_case_dataframe.index)

print(f"total case : {tgt_cnt}\n")

print("청약 후 상장 당일 시가 매도")
print(f"win  : {win_cnt} ({win_cnt / tgt_cnt * 100:.2f}% / {win_case_dataframe.mean():.2f}%)")
print(f"lose : {lose_cnt} ({lose_cnt / tgt_cnt * 100:.2f}% / {lose_case_dataframe.mean():.2f}%)")
print(f"=> {ret.mean():.2f}%\n")

total case : 737

청약 후 상장 당일 시가 매도
win  : 587 (79.65% / 39.33%)
lose : 150 (20.35% / -6.12%)
=> 30.08%



### 청약 후 상장 당일 종가 매도

In [ ]:
ret = stock_info_dataframe['첫날종가/공모가(%)']

tgt_cnt = len(ret.index)
win_case_dataframe = ret[ret >= 0]
lose_case_dataframe = ret[ret < 0]
win_cnt = len(win_case_dataframe.index)
lose_cnt = len(lose_case_dataframe.index)

print(f"total case : {tgt_cnt}\n")

print("청약 후 상장 당일 종가 매도")
print(f"win  : {win_cnt} ({win_cnt / tgt_cnt * 100:.2f}% / {win_case_dataframe.mean():.2f}%)")
print(f"lose : {lose_cnt} ({lose_cnt / tgt_cnt * 100:.2f}% / {lose_case_dataframe.mean():.2f}%)")
print(f"=> {ret.mean():.2f}%\n")

total case : 737

청약 후 상장 당일 종가 매도
win  : 558 (75.71% / 43.76%)
lose : 179 (24.29% / -10.74%)
=> 30.53%



## 기관경쟁률 1000:1 이상 종목


In [ ]:
FILE_PATH = '/content/gdrive/Shareddrives/CS360/madcamp-3-week/data/strategy-2/strategy-2-data.csv'
stock_info_dataframe = pd.read_csv(FILE_PATH, index_col = '기업명', thousands=',')

In [ ]:
stock_info_dataframe

,신규상장일,현재가(원),전일비(%),공모가(원),공모가대비등락률(%),시초가(원),시초/공모(%),첫날종가(원),기관경쟁률
기업명,,,,,,,,,
마스턴프리미어리츠(유가),2022/05/31,"5,010",0.00%,5000,0.2%,5750,15%,6010,NaN
대명에너지,2022/05/16,"12,550",-5.28%,15000,-16.33%,15450,3%,13950,254.74
상상인스팩3호,2022/04/29,"2,115",-0.24%,2000,5.75%,2155,7.75%,2150,NaN
신영스팩7호,2022/04/28,"2,170",0.93%,2000,8.5%,2295,14.75%,2270,NaN
포바이포,2022/04/28,"23,750",13.10%,17000,39.71%,34000,100%,44200,1846.32
...,...,...,...,...,...,...,...,...,...
아이티센,2014/12/23,-,%,7500,%,8460,12.8%,8770,54.13
제일모직(구.삼성에버랜드)(유가),2014/12/18,-,%,53000,%,106000,100%,113000,465.00
하이로닉,2014/12/17,-,%,51000,%,63800,25.1%,73300,500.56


In [ ]:
# 기관 경쟁률 1000:1 이상인 종목만 필터링
stock_info_dataframe = stock_info_dataframe.loc[stock_info_dataframe['기관경쟁률'].dropna().index]

In [ ]:
stock_info_dataframe

,신규상장일,현재가(원),전일비(%),공모가(원),공모가대비등락률(%),시초가(원),시초/공모(%),첫날종가(원),기관경쟁률
기업명,,,,,,,,,
대명에너지,2022/05/16,"12,550",-5.28%,15000,-16.33%,15450,3%,13950,254.74
포바이포,2022/04/28,"23,750",13.10%,17000,39.71%,34000,100%,44200,1846.32
지투파워,2022/04/01,"33,550",-7.19%,16400,104.57%,32800,100%,35250,1729.63
세아메카닉스,2022/03/24,"6,600",3.45%,4400,50%,8800,100%,7600,1812.83
공구우먼,2022/03/23,"14,250",-25.59%,20000,-28.75%,21600,8%,20000,56.90
...,...,...,...,...,...,...,...,...,...
아스트,2014/12/24,-,%,9500,%,8550,-10%,9000,74.59
넥스트엔터테인먼트월드,2014/12/23,-,%,16300,%,15800,-3.07%,18150,214.69
아이티센,2014/12/23,-,%,7500,%,8460,12.8%,8770,54.13


In [ ]:
stock_info_dataframe = stock_info_dataframe[stock_info_dataframe['기관경쟁률'] >= 1000]
stock_info_dataframe = stock_info_dataframe.loc[:, ['공모가(원)', '시초가(원)', '첫날종가(원)']]

In [ ]:
stock_info_dataframe

,공모가(원),시초가(원),첫날종가(원)
기업명,,,
포바이포,17000,34000,44200
지투파워,16400,32800,35250
세아메카닉스,4400,8800,7600
유일로보틱스,10000,20000,26000
비씨엔씨,13000,26000,22600
...,...,...,...
윌링스,12500,22800,29600
대모(구.대모엔지니어링),5200,8710,11300
플리토,26000,31600,27800


In [ ]:
stock_info_dataframe['시초가/공모가(%)'] = (stock_info_dataframe['시초가(원)']/stock_info_dataframe['공모가(원)'] - 1)*100
stock_info_dataframe['첫날종가/공모가(%)'] = (stock_info_dataframe['첫날종가(원)']/stock_info_dataframe['공모가(원)'] - 1)*100
stock_info_dataframe

,공모가(원),시초가(원),첫날종가(원),시초가/공모가(%),첫날종가/공모가(%)
기업명,,,,,
포바이포,17000,34000,44200,100.000000,160.000000
지투파워,16400,32800,35250,100.000000,114.939024
세아메카닉스,4400,8800,7600,100.000000,72.727273
유일로보틱스,10000,20000,26000,100.000000,160.000000
비씨엔씨,13000,26000,22600,100.000000,73.846154
...,...,...,...,...,...
윌링스,12500,22800,29600,82.400000,136.800000
대모(구.대모엔지니어링),5200,8710,11300,67.500000,117.307692
플리토,26000,31600,27800,21.538462,6.923077


### 청약 후 상장 당일 시가 매도

In [ ]:
ret = stock_info_dataframe['시초가/공모가(%)']

tgt_cnt = len(ret.index)
win_case_dataframe = ret[ret >= 0]
lose_case_dataframe = ret[ret < 0]
win_cnt = len(win_case_dataframe.index)
lose_cnt = len(lose_case_dataframe.index)

print(f"total case : {tgt_cnt}\n")

print("청약 후 상장 당일 시가 매도")
print(f"win  : {win_cnt} ({win_cnt / tgt_cnt * 100:.2f}% / {win_case_dataframe.mean():.2f}%)")
print(f"lose : {lose_cnt} ({lose_cnt / tgt_cnt * 100:.2f}% / {lose_case_dataframe.mean():.2f}%)")
print(f"=> {ret.mean():.2f}%\n")

total case : 140

청약 후 상장 당일 시가 매도
win  : 135 (96.43% / 74.11%)
lose : 5 (3.57% / -4.06%)
=> 71.32%



### 청약 후 상장 종가 매도

In [ ]:
ret = stock_info_dataframe['첫날종가/공모가(%)']

tgt_cnt = len(ret.index)
win_case_dataframe = ret[ret >= 0]
lose_case_dataframe = ret[ret < 0]
win_cnt = len(win_case_dataframe.index)
lose_cnt = len(lose_case_dataframe.index)

print(f"total case : {tgt_cnt}\n")

print("청약 후 상장 당일 종가 매도")
print(f"win  : {win_cnt} ({win_cnt / tgt_cnt * 100:.2f}% / {win_case_dataframe.mean():.2f}%)")
print(f"lose : {lose_cnt} ({lose_cnt / tgt_cnt * 100:.2f}% / {lose_case_dataframe.mean():.2f}%)")
print(f"=> {ret.mean():.2f}%\n")

total case : 140

청약 후 상장 당일 종가 매도
win  : 133 (95.00% / 81.06%)
lose : 7 (5.00% / -7.14%)
=> 76.65%

